In [1]:
import re
import os
import shutil

In [31]:
class OpenVersionedFile:
    
    MAX_VERSIONS = 3
    
    def __init__(self, filename, mode='r'):
        self.path, self.filename= os.path.split(filename)
        if not self.path:
            self.path = "."
        self.full_path = os.path.join(self.path, self.filename)
        self.base, self.extension = os.path.splitext(self.filename)
        self.extension = self.extension[1:]  # Get rid of the dot
        self.pat = re.compile(f'{self.base}.~(\\d+).{self.extension}')
        if 'a' in mode or 'w' in mode:
            self.make_backup()
        self.file_handler = open(self.full_path, mode)        

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()
        
    def write(self, s):
        self.file_handler.write(s)
        
    def read(self, size=-1):
        return self.file_handler.read(size)
    
    def close(self):
        self.file_handler.close()
        
    def get_versioned_name(self, num):
        return f"{self.base}.~{num}.{self.extension}"
    
    def make_backup(self):
        if os.path.isfile(self.full_path):
            print(f'start backup of {self.full_path}')
            copies = self.get_last_copies()
            if copies:
                copies = self.shift(copies)
            new_filename = self.get_versioned_name(1)
            print(f"-> os copy {self.full_path} -> {new_filename}")
            shutil.copy(self.full_path, new_filename)
               
    def shift(self, copies):
        result = []
        while len(copies) >= self.MAX_VERSIONS:
            (_, filename) = copies.pop(0)
            print(f"-> delete {filename}")
            os.remove(filename)
        for num, filename in copies:
            new_filename = self.get_versioned_name(num+1)
            result.append((num+1, new_filename))
            print(f"-> os rename {filename} -> {new_filename}")
            shutil.move(filename, new_filename)
        return result
        
    def get_last_copies(self):
        seq = (
            tuple([self.pat.match(f), f])
            for f in os.listdir(self.path)
        )
        copies = [(int(m.group(1)), fn) for m, fn in seq if m]
        return list(reversed(sorted(copies)))

In [32]:
! rm hola*txt

f = OpenVersionedFile("hola.txt", 'w')
f.write("Esta es la primera versión")
f.close()
        
f = OpenVersionedFile("hola.txt", 'w')
f.write("Esta es la segunda versión")
f.close()

f = OpenVersionedFile("hola.txt", 'w')
f.write("Esta es la tercera versión")
f.close()

f = OpenVersionedFile("hola.txt", 'w')
f.write("Esta es la cuarta versión")
f.close()

with OpenVersionedFile("hola.txt", 'w') as f:
    f.write("Esta es la quinta versión")



!ls hola*

start backup of ./hola.txt
-> os copy ./hola.txt -> hola.~1.txt
start backup of ./hola.txt
-> os rename hola.~1.txt -> hola.~2.txt
-> os copy ./hola.txt -> hola.~1.txt
start backup of ./hola.txt
-> os rename hola.~2.txt -> hola.~3.txt
-> os rename hola.~1.txt -> hola.~2.txt
-> os copy ./hola.txt -> hola.~1.txt
start backup of ./hola.txt
-> delete hola.~3.txt
-> os rename hola.~2.txt -> hola.~3.txt
-> os rename hola.~1.txt -> hola.~2.txt
-> os copy ./hola.txt -> hola.~1.txt
hola.txt    hola.~1.txt hola.~2.txt hola.~3.txt


In [33]:
cat hola.txt

Esta es la quinta versión

In [37]:
cat hola.~4.txt

cat: hola.~4.txt: No such file or directory


In [38]:
with OpenVersionedFile("hola.txt", 'r') as f:
    print(f.read())

Esta es la quinta versión


In [39]:
!ls hola*

hola.txt    hola.~1.txt hola.~2.txt hola.~3.txt


In [40]:
!cat hola.txt

Esta es la quinta versión

In [41]:
!cat hola.~1.txt

Esta es la cuarta versión

In [42]:
!cat hola.~2.txt

Esta es la tercera versión

In [43]:
!cat hola.~3.txt

Esta es la segunda versión

In [49]:
!rm aa*.txt && echo "First line" > aa.txt && ls aa*

aa.txt


In [50]:
with OpenVersionedFile('aa.txt', 'a') as f:
    f.write("Second line")

start backup of ./aa.txt
-> os copy ./aa.txt -> aa.~1.txt


In [51]:
!cat aa.txt

First line
Second line

In [52]:
!cat aa.~1.txt

First line
